In [1]:
from huggingface_hub import notebook_login

notebook_login()
repo_name='wav2vec2-large-xlsr-sw-ASR'

In [2]:
from google.colab import drive
import os

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
model_path = "facebook/wav2vec2-large-xlsr-53"
BASE = '/content/drive/MyDrive/ASR_wav2vec_project/preprocessing/processed_data/14.0-delta-2023-06-23'
CSV = f'{BASE}/manifest_sw_14_0_delta.csv'
AUDIO_DIR = f'{BASE}/cleaned_sw_audio_14_0_delta'
# If you're using a pre-configured HuggingFace processor, you can comment out or remove VOCAB
VOCAB = f'{BASE}/vocab.json'

In [4]:
import pandas as pd
df = pd.read_csv(CSV)
# df['wav_path'] = df['wav_path'].apply(lambda p: os.path.join(AUDIO_DIR, p.split('\\')[-1]))
df["wav_path"] = df["wav_path"].apply(lambda p: os.path.join(AUDIO_DIR, os.path.basename(p)))
print(f"Total samples in dataset: {len(df)}")

Total samples in dataset: 270


In [5]:
df.keys()

Index(['wav_path', 'duration', 'transcript'], dtype='object')

In [6]:
from datasets import Dataset, Audio
dataset = Dataset.from_pandas(df[['wav_path', 'transcript']])

In [7]:
import re

def remove_special_characters(batch, column_names='transcript'):
  chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\(\)]'
  batch[column_names] = re.sub(chars_to_ignore_regex, '', batch[column_names]).lower() + " "
  return batch

dataset = dataset.map(remove_special_characters)

print(len(dataset['wav_path']))

Map:   0%|          | 0/270 [00:00<?, ? examples/s]

270


In [8]:
import re

def extract_all_chars(batch, column_names='transcript'):
  all_text = " ".join(batch[column_names])
  vocab = list(set(all_text))
  # Return a list of vocabularies and all_text, with length equal to the batch size
  batch_size = len(batch[column_names])
  return {"vocab": [vocab] * batch_size, "all_text": [all_text] * batch_size}

In [9]:
vocabs = dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset.column_names)

Map:   0%|          | 0/270 [00:00<?, ? examples/s]

In [10]:
vocab_list = list(set(vocabs['vocab'][0]))
vocab_dict ={v: k for k, v in enumerate(vocab_list)}
vocab_dict['|'] = vocab_dict[' ']
del vocab_dict[' ']
vocab_dict['[UNK]'] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
vocab_dict

{'p': 0,
 'w': 1,
 'u': 2,
 'b': 3,
 'k': 4,
 'l': 5,
 's': 6,
 'y': 7,
 'n': 8,
 'j': 9,
 'o': 11,
 't': 12,
 'e': 13,
 'd': 14,
 'h': 15,
 'i': 16,
 'f': 17,
 'm': 18,
 'c': 19,
 'r': 20,
 'a': 21,
 'g': 22,
 'z': 23,
 'v': 24,
 'q': 25,
 '|': 10,
 '[UNK]': 26,
 '[PAD]': 27}

In [11]:
import json
with open(VOCAB, 'w') as vocab_file:
  json.dump(vocab_dict, vocab_file)

In [12]:
from transformers import Wav2Vec2CTCTokenizer
tokenizer = Wav2Vec2CTCTokenizer(VOCAB,
                                 unk_token="[UNK]",
                                 pad_token="[PAD]",
                                 word_delimiter_token="|")

In [13]:
tokenizer.push_to_hub(repo_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jkhyjkhy/wav2vec2-large-xlsr-sw-ASR/commit/278be9fa75f81a9fdaf6fc5f01a0ee8010333922', commit_message='Upload tokenizer', commit_description='', oid='278be9fa75f81a9fdaf6fc5f01a0ee8010333922', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jkhyjkhy/wav2vec2-large-xlsr-sw-ASR', endpoint='https://huggingface.co', repo_type='model', repo_id='jkhyjkhy/wav2vec2-large-xlsr-sw-ASR'), pr_revision=None, pr_num=None)

In [14]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1,
                                             sampling_rate=16000,
                                             padding_value=0.0,
                                             do_normalize=True,
                                             return_attention_mask=False)

In [15]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor,
                              tokenizer=tokenizer)

In [18]:
# Split the dataset first
dataset = dataset.train_test_split(test_size=0.1)
dataset = dataset.cast_column("wav_path", Audio(sampling_rate=16000))

def prepare_dataset(batch):
    audio = batch["wav_path"]

    audio_inputs = processor(audio["array"], sampling_rate=audio["sampling_rate"])

    batch["input_values"] = audio_inputs["input_values"][0]
    batch["input_length"] = len(batch["input_values"])
    label_inputs = processor.tokenizer(batch["transcript"])
    batch["labels"] = label_inputs["input_ids"]

    return batch

In [19]:
embeded_dataset = dataset.map(prepare_dataset, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/243 [00:00<?, ? examples/s]

Map:   0%|          | 0/27 [00:00<?, ? examples/s]

In [20]:
embeded_dataset

DatasetDict({
    train: Dataset({
        features: ['input_values', 'input_length', 'labels'],
        num_rows: 243
    })
    test: Dataset({
        features: ['input_values', 'input_length', 'labels'],
        num_rows: 27
    })
})

In [21]:
import IPython.display as ipd
import numpy as np
import random

# Checking stage
# Use the original dataset for audio playback
# Use the embeded_dataset for checking processed data
rand_int = random.randint(0, len(dataset['train']))
print(f"Original transcript: {dataset['train']['transcript'][rand_int]}")
print(f"Processed transcript: {embeded_dataset['train']['labels'][rand_int]}")

# Play the audio from the original dataset
ipd.Audio(data=dataset['train']['wav_path'][rand_int]['array'], autoplay=True, rate=16000)

# Check the processed data in embeded_dataset
print("\nChecking embeded_dataset sample:")
print(f"Input values shape: {len(embeded_dataset['train']['input_values'][rand_int])}")
print(f"Input length: {embeded_dataset['train']['input_length'][rand_int]}")
print(f"Labels: {embeded_dataset['train']['labels'][rand_int]}")

Original transcript: serikali ya rwanda vile vile imedai kwamba watu hao wawili walioasilishwa 
Processed transcript: [6, 13, 20, 16, 4, 21, 5, 16, 10, 7, 21, 10, 20, 1, 21, 8, 14, 21, 10, 24, 16, 5, 13, 10, 24, 16, 5, 13, 10, 16, 18, 13, 14, 21, 16, 10, 4, 1, 21, 18, 3, 21, 10, 1, 21, 12, 2, 10, 15, 21, 11, 10, 1, 21, 1, 16, 5, 16, 10, 1, 21, 5, 16, 11, 21, 6, 16, 5, 16, 6, 15, 1, 21, 10]

Checking embeded_dataset sample:
Input values shape: 104416
Input length: 104416
Labels: [6, 13, 20, 16, 4, 21, 5, 16, 10, 7, 21, 10, 20, 1, 21, 8, 14, 21, 10, 24, 16, 5, 13, 10, 24, 16, 5, 13, 10, 16, 18, 13, 14, 21, 16, 10, 4, 1, 21, 18, 3, 21, 10, 1, 21, 12, 2, 10, 15, 21, 11, 10, 1, 21, 1, 16, 5, 16, 10, 1, 21, 5, 16, 11, 21, 6, 16, 5, 16, 6, 15, 1, 21, 10]


In [57]:
processor.push_to_hub(repo_name)

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jkhyjkhy/wav2vec2-large-xlsr-sw-ASR/commit/0397bc318ee862911f47cddb602276ffb7c66b24', commit_message='Upload processor', commit_description='', oid='0397bc318ee862911f47cddb602276ffb7c66b24', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jkhyjkhy/wav2vec2-large-xlsr-sw-ASR', endpoint='https://huggingface.co', repo_type='model', repo_id='jkhyjkhy/wav2vec2-large-xlsr-sw-ASR'), pr_revision=None, pr_num=None)

### Training stage

In [22]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [23]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [24]:
!pip install evaluate jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 56.6 MB/s eta 0:00:00


In [25]:
import evaluate
wer_metric = evaluate.load('wer')

In [26]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [27]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    model_path,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)
model.freeze_feature_extractor()

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1857: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [40]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=4,
  gradient_accumulation_steps=2,
  eval_strategy="steps",
  num_train_epochs=15,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=50,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=0,
  save_total_limit=2,
  push_to_hub=True,
  report_to="none", # Explicitly disable reporting to services like wandb
)

In [41]:
!wandb disabled

W&B disabled.


In [42]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=embeded_dataset["train"],
    eval_dataset=embeded_dataset["test"],
    tokenizer=processor.feature_extractor,
)

/tmp/ipython-input-42-3875103141.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [43]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:170: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
100,2.832900,3.101232,1.000000
200,2.821400,3.018268,1.000000
300,2.823400,2.956967,1.000000
400,2.770000,2.864543,1.000000


/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:170: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:170: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:170: User

TrainOutput(global_step=465, training_loss=2.8072012788505964, metrics={'train_runtime': 389.1333, 'train_samples_per_second': 9.367, 'train_steps_per_second': 1.195, 'total_flos': 5.114519377974989e+17, 'train_loss': 2.8072012788505964, 'epoch': 15.0})

In [44]:
trainer.push_to_hub()

Uploading...:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jkhyjkhy/wav2vec2-large-xlsr-sw-ASR/commit/6192dc2f4eecde77a638ef41b6b1113c2a500c2f', commit_message='End of training', commit_description='', oid='6192dc2f4eecde77a638ef41b6b1113c2a500c2f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jkhyjkhy/wav2vec2-large-xlsr-sw-ASR', endpoint='https://huggingface.co', repo_type='model', repo_id='jkhyjkhy/wav2vec2-large-xlsr-sw-ASR'), pr_revision=None, pr_num=None)

### Evaluate

In [60]:
# Use the existing tokenizer and feature_extractor to create the processor
processor_eval = Wav2Vec2Processor.from_pretrained(repo_name)
model = Wav2Vec2ForCTC.from_pretrained(repo_name)

In [75]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"]).unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  print(pred_ids)
  batch["pred_str"] = processor_eval.batch_decode(pred_ids)[0]
  batch["text"] = processor_eval.decode(batch["labels"], group_tokens=False)

  return batch

In [76]:
results = embeded_dataset["test"].map(map_to_result, remove_columns=embeded_dataset["test"].column_names)
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Map:   0%|          | 0/27 [00:00<?, ? examples/s]

tensor([[27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
         27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
         27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
         27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
         27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
         27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
         27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
         27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
         27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
         27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
         27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
         27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
         27, 27, 27, 27, 27,

In [65]:
from IPython.display import display, HTML
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [72]:
show_random_elements(results)

,pred_str,text
0,,kundi la dola ya kiislamu lilidai kuwa wanachama wake waliwaua takribani wakristo ishirini
1,,udongo wenye magadi
2,,jinsi ya kuanda viazi lishe
3,,vitunguu swaumu vilivyo sagwa
4,,tatu kwa mwezi huo juu zaidi kutoka dola milioni ishirini na tisa
5,,ushindani ni mkali
6,,dalili ya ugonjwa wa ndama kuhara ni macho ya mnyama kuingia ndani
7,,tenga pembeni kwa dakika ishirini
8,,jinsi ya kutumia samadi kwa ubora wake
9,,ongeza vitunguu swaumu na kitunguu


In [78]:
tokenizer.decode(embeded_dataset["test"]['labels'][0])

'kampala inasafirisha bidha kwenda jamuhuri ya kidemokrasia ya kongo'